# National Oceanic and Atmospheric Administration (NOAA)
This jupyter notebook is meant to be used along with the North American Mesoscale Forecast System (NAM) dataset.  
This dataset can be found online under "Data Access > Model > Datasets > NAM" on the NOAA website.  
Once the data has been properly requested, confirmed, and processed using NOAA's Order Data feature,  
the requester is given 5 days to download the data via a email-link.  

# NAM 2017
To be more specific, the range this notebook will be targetting is the entire 2017 year (1800 UTC only). (100 ish Gb)  
Through further observation of the email-link provided by NOAA, it can be seen that the files end in an extension ".tar". These are all just zipped files. BEWARE- unpacking all these files with the following command doubles the 100Gb to 200Gb.  
It is recommended to make a main folder (moddata) with subfolders with each month on them (01, 02, 03, ..., 12).  
This way makes everything easier. The idea is to unpack an entire month and then delete the ".tar" files for that month.  
That way you don't have to unpack 100Gb to 200Gb and then try to delete 100Gb.

# Unpacking ".tar" Files
The ".tar" file names should look like the following:

namanl_218_2017010118.g2.tar

The format is very simple: namanl_218_yyyymmddhh.g2.tar  
Where yyyy = year, mm = month, dd = day, hh = hour (UTC)

Since this data ranges accross the entire North America continent, 18 UTC was chosen. In New York, 18 UTC translates to 2 P.M. This makes any "real images" from the dataset appear more visually appealing since that part of the Earth will be facing towards the sun. This would be better than "real images" of NA taken at night. This also saves a lot of space. Imagine simply having 2 timestamps per day instead of 1, this would easily double the size of the data.

In [ ]:
import pygrib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def print_grbs(grbs):
    """
    prints the contents of a pygrib.open(filename) variable
    """
    for grb in grbs:
        print(grb)

In [ ]:
satdata_2017_01_01_00_0 = "mod/moddata/nam_218_20170101_0000_000.grb2"
grbs = pygrib.open(satdata_2017_01_01_00_0)
print(grbs)

In [ ]:
print_grbs(grbs)

In [ ]:
temp = grbs.select(name="Temperature")[0].values

In [ ]:
print(temp)

In [ ]:
plt.figure()
ax = sns.heatmap(temp, cbar='true')
ax.invert_yaxis()